In [1]:
import numpy as np
np.set_printoptions(precision = 4, linewidth = 100, suppress = True)

In [93]:
np.random.seed(4)
size = 100
data = np.random.poisson(lam=10, size=(size, 20)).astype(np.float32)
data -= data.mean(axis=0)[None, :]
cov = np.cov(data.T)

In [94]:
evals, evecs = np.linalg.eigh(cov[:2, :2])

In [95]:
evecs.dot(cov[:2, :2]).dot(evecs.T)

array([[ 8.1284,  0.    ],
       [ 0.    , 10.7814]])

In [98]:
np.square(np.linalg.svd(data[:, :2], full_matrices=False)[0][:, 1].dot(
    (data[:, :2].dot(evecs) / np.sqrt(evals[None, :]))[:, 0]
))

98.99999986885754

In [99]:
vec_proj = (data[:, :2].dot(evecs) / np.sqrt(evals[None, :])).T.dot(
    data[:, 2:3]
) / (size - 1)
vec_proj

array([[-0.7572],
       [ 0.3132]])

In [107]:
# Much more useful than holding the raw data
np.allclose(
    (evecs.T / np.sqrt(evals[:, None])).dot(cov[0:2, 2:3]),
    vec_proj,
    rtol=1e-17)

True

In [108]:
list(((evecs.T / np.sqrt(evals[:, None])).dot(cov[0:2, 2:3]) - vec_proj)[:, 0])

[3.3306690738754696e-16, -5.551115123125783e-17]

In [109]:
data[:, 2].dot(data[:, 2]) / (size-1) == cov[2, 2]

True

In [111]:
np.sqrt(cov[2, 2] - np.linalg.norm(vec_proj) ** 2)

3.467777587428768

In [112]:
# A^T A and A A^T have the same eigenvalues and trace, also constructed
# so that A A^T is similar to the augmented covariance matrix - where the
# update to the trace is precisely cov[2, 2].
A = np.asarray(
    [[np.sqrt(evals[0]), 0, vec_proj[0, 0]],
     [0, np.sqrt(evals[1]), vec_proj[1, 0]],
     [0, 0, np.sqrt(cov[2, 2] - np.linalg.norm(vec_proj) ** 2)]],
)
A

array([[ 2.851 ,  0.    , -0.7572],
       [ 0.    ,  3.2835,  0.3132],
       [ 0.    ,  0.    ,  3.4678]])

In [113]:
np.linalg.eigh(A.dot(A.T))[0]

array([ 7.2273, 10.5245, 13.855 ])

In [114]:
np.linalg.eigh(cov[:3, :3])[0]

array([ 7.2273, 10.5245, 13.855 ])

In [115]:
np.allclose(
    np.linalg.eigh(A.dot(A.T))[0],
    np.linalg.eigh(cov[:3, :3])[0],
    rtol=1e-10)

True